# Exploring `statsapi`

We found a Python wrapper for the MLB Gameday API that might help us get the pitch data we need. Here are some links.  

Repo:  
https://github.com/toddrob99/MLB-StatsAPI

Doc:  
https://toddrob99.github.io/MLB-StatsAPI/#statsapi.schedule

In [1]:
!pwd

/Users/werlindo/Dropbox/flatiron/mod_3_proj/dsc-3-final-project


### Libraries

In [1]:
import statsapi
from statsapi import ENDPOINTS

---

Let's look at the `ENDPOINTS` to see if we can figure out we can get.

In [2]:
ENDPOINTS

{'attendance': {'url': 'https://statsapi.mlb.com/api/{ver}/attendance',
  'path_params': {'ver': {'type': 'str',
    'default': 'v1',
    'leading_slash': False,
    'trailing_slash': False,
    'required': True}},
  'query_params': ['teamId',
   'leagueId',
   'season',
   'date',
   'leagueListId',
   'gameType',
   'fields'],
  'required_params': [['teamId'], ['leagueId'], ['leagueListid']]},
 'awards': {'url': 'https://statsapi.mlb.com/api/{ver}/awards{awardId}{recipients}',
  'path_params': {'ver': {'type': 'str',
    'default': 'v1',
    'leading_slash': False,
    'trailing_slash': False,
    'required': True},
   'awardId': {'type': 'str',
    'default': None,
    'leading_slash': True,
    'trailing_slash': False,
    'required': False},
   'recipients': {'type': 'bool',
    'default': True,
    'True': '/recipients',
    'False': '',
    'leading_slash': False,
    'trailing_slash': False,
    'required': False}},
  'query_params': ['sportId', 'leagueId', 'season', 'hydrate',

( I cleared the output above because it was very large)  
On visually inspecting, we can see some promising features in there, e.g. `spin_rate` etc.  

But need to figure out how to navigate down to what we need. Maybe playing with some examples from the documentation will help spark some ideas.

Here's an example from the documentation:

In [185]:
print( statsapi.player_stats(next(x['id'] for x in statsapi.get('sports_players',{'season':2008,'gameType':'W'})['people'] if x['fullName']=='Chase Utley'), 'hitting', 'career') )

Chase "Silver Fox" Utley, 2B (2003-2018)

Career Hitting
gamesPlayed: 1937
groundOuts: 1792
runs: 1103
doubles: 411
triples: 58
homeRuns: 259
strikeOuts: 1193
baseOnBalls: 724
intentionalWalks: 62
hits: 1885
hitByPitch: 204
avg: .275
atBats: 6857
obp: .358
slg: .465
ops: .823
caughtStealing: 22
stolenBases: 154
groundIntoDoublePlay: 93
numberOfPitches: 31043
plateAppearances: 7863
totalBases: 3189
rbi: 1025
leftOnBase: 2780
sacBunts: 6
sacFlies: 72
babip: .297
groundOutsToAirouts: 0.84




...And another example:

In [ ]:
statsapi.get('game_timestamps',{'gamePk':565997})

(Cleared the output above because it is too large to have rendered)  
Hmmm. This is promising. We saw `game_playByPlay` in the `ENDPOINTS`, and think we just need a `gamePk` 

In [3]:
test_game = statsapi.get('game_playByPlay',{'gamePk':565997})

In [208]:
type(test_game)

dict

In [11]:
test_game.keys()

dict_keys(['copyright', 'allPlays', 'currentPlay', 'scoringPlays', 'playsByInning'])

In [13]:
all_plays = test_game['allPlays']

In [82]:
all_plays[0]

{'result': {'type': 'atBat',
  'event': 'Strikeout',
  'eventType': 'strikeout',
  'description': 'Andrew McCutchen strikes out swinging.',
  'rbi': 0,
  'awayScore': 0,
  'homeScore': 0},
 'about': {'atBatIndex': 0,
  'halfInning': 'top',
  'inning': 1,
  'startTime': '2019-04-24T22:58:20.000Z',
  'endTime': '2019-04-24T23:13:04.000Z',
  'isComplete': True,
  'isScoringPlay': False,
  'hasReview': False,
  'hasOut': True,
  'captivatingIndex': 14},
 'count': {'balls': 3, 'strikes': 3, 'outs': 1},
 'matchup': {'batter': {'id': 457705,
   'fullName': 'Andrew McCutchen',
   'link': '/api/v1/people/457705'},
  'batSide': {'code': 'R', 'description': 'Right'},
  'pitcher': {'id': 450306,
   'fullName': 'Jason Vargas',
   'link': '/api/v1/people/450306'},
  'pitchHand': {'code': 'L', 'description': 'Left'},
  'batterHotColdZones': [],
  'pitcherHotColdZones': [],
  'splits': {'batter': 'vs_LHP', 'pitcher': 'vs_RHB', 'menOnBase': 'Empty'}},
 'pitchIndex': [0, 1, 2, 3, 4, 5],
 'actionIndex': 

In [88]:
type(all_plays[0])

dict

In [87]:
all_plays[0].keys()

dict_keys(['result', 'about', 'count', 'matchup', 'pitchIndex', 'actionIndex', 'runnerIndex', 'runners', 'playEvents', 'atBatIndex', 'playEndTime'])

In [84]:
type(all_plays[0].get('playEvents'))

list

In [27]:
pitch_num = all_plays[0].get('playEvents')[1].get('pitchNumber')
pitch_num

2

In [43]:
type(all_plays)

list

In [45]:
df = pd.DataFrame(all_plays)

In [ ]:
df.columns()

In [46]:
df.head()

about actionIndex  atBatIndex  \
0  {'atBatIndex': 0, 'halfInning': 'top', 'inning...          []           0   
1  {'atBatIndex': 1, 'halfInning': 'top', 'inning...          []           1   
2  {'atBatIndex': 2, 'halfInning': 'top', 'inning...          []           2   
3  {'atBatIndex': 3, 'halfInning': 'top', 'inning...          []           3   
4  {'atBatIndex': 4, 'halfInning': 'top', 'inning...          []           4   

                                   count  \
0  {'balls': 3, 'strikes': 3, 'outs': 1}   
1  {'balls': 2, 'strikes': 2, 'outs': 1}   
2  {'balls': 2, 'strikes': 0, 'outs': 1}   
3  {'balls': 4, 'strikes': 2, 'outs': 1}   
4  {'balls': 0, 'strikes': 0, 'outs': 2}   

                                             matchup  \
0  {'batter': {'id': 457705, 'fullName': 'Andrew ...   
1  {'batter': {'id': 592663, 'fullName': 'J.T. Re...   
2  {'batter': {'id': 547180, 'fullName': 'Bryce H...   
3  {'batter': {'id': 656555, 'fullName': 'Rhys Ho...   
4  {'batter': {'id': 596748, 'fullName': 'Maikel ...   

                 pitchIndex               playEndTime  \
0        [0, 1, 2, 3, 4, 5]  2019-04-24T23:13:04.000Z   
1  [0, 1, 2, 3, 4, 5, 6, 7]  2019-04-24T23:15:53.000Z   
2                 [0, 1, 2]  2019-04-24T23:17:43.000Z   
3     [0, 1, 2, 3, 4, 5, 6]  2019-04-24T23:21:06.000Z   
4                       [0]  2019-04-24T23:21:54.000Z   

                                          playEvents  \
0  [{'details': {'call': {'code': 'B', 'descripti...   
1  [{'details': {'call': {'code': 'S', 'descripti...   
2  [{'details': {'call': {'code': 'B', 'descripti...   
3  [{'details': {'call': {'code': 'B', 'descripti...   
4  [{'details': {'call': {'code': 'X', 'descripti...   

                                              result runnerIndex  \
0  {'type': 'atBat', 'event': 'Strikeout', 'event...         [0]   
1  {'type': 'atBat', 'event': 'Double', 'eventTyp...         [0]   
2  {'type': 'atBat', 'event': 'Double', 'eventTyp...   [0, 1, 2]   
3  {'type': 'atBat', 'event': 'Walk', 'eventType'...         [0]   
4  {'type': 'atBat', 'event': 'Flyout', 'eventTyp...         [0]   

                                             runners  
0  [{'movement': {'start': None, 'end': None, 'ou...  
1  [{'movement': {'start': None, 'end': '2B', 'ou...  
2  [{'movement': {'start': '2B', 'end': '3B', 'ou...  
3  [{'movement': {'start': None, 'end': '1B', 'ou...  
4  [{'movement': {'start': None, 'end': None, 'ou...

In [40]:
df = pd.DataFrame.from_dict(test_game)


ValueError: arrays must all be same length

In [184]:
len(test_game)

5

- each dictionary in `all_plays` is a single 'play'

In [125]:
print(json.dumps(all_plays[-1], indent=4))

{
    "result": {
        "type": "atBat",
        "event": "Strikeout",
        "eventType": "strikeout",
        "description": "J.D. Davis strikes out swinging.",
        "rbi": 0,
        "awayScore": 6,
        "homeScore": 0
    },
    "about": {
        "atBatIndex": 79,
        "halfInning": "bottom",
        "inning": 9,
        "startTime": "2019-04-25T02:30:32.000Z",
        "endTime": "2019-04-25T02:32:40.000Z",
        "isComplete": true,
        "isScoringPlay": false,
        "hasReview": false,
        "hasOut": true,
        "captivatingIndex": 14
    },
    "count": {
        "balls": 3,
        "strikes": 3,
        "outs": 3
    },
    "matchup": {
        "batter": {
            "id": 605204,
            "fullName": "J.D. Davis",
            "link": "/api/v1/people/605204"
        },
        "batSide": {
            "code": "R",
            "description": "Right"
        },
        "pitcher": {
            "id": 504379,
            "fullName": "Juan Nicasio",
     

In [112]:
test_game.keys()

dict_keys(['copyright', 'allPlays', 'currentPlay', 'scoringPlays', 'playsByInning'])

### So just on inspection, I think the meat of what we need is in `allPlays`

I'm curious, though, as to what is in `playsByInning`

In [113]:
test_game.get('playsByInning')

[{'startIndex': 0,
  'endIndex': 11,
  'top': [0, 1, 2, 3, 4, 5],
  'bottom': [6, 7, 8, 9, 10, 11],
  'hits': {'away': [{'team': {'id': 143,
      'name': 'Philadelphia Phillies',
      'link': '/api/v1/teams/143',
      'springLeague': {'id': 115,
       'name': 'Grapefruit League',
       'link': '/api/v1/league/115',
       'abbreviation': 'GL'},
      'allStarStatus': 'N'},
     'inning': 1,
     'pitcher': {'id': 450306,
      'fullName': 'Jason Vargas',
      'link': '/api/v1/people/450306'},
     'batter': {'id': 592663,
      'fullName': 'J.T. Realmuto',
      'link': '/api/v1/people/592663'},
     'coordinates': {'x': 167.5117925095603, 'y': 55.70531840499331},
     'type': 'H',
     'description': 'Double'},
    {'team': {'id': 143,
      'name': 'Philadelphia Phillies',
      'link': '/api/v1/teams/143',
      'springLeague': {'id': 115,
       'name': 'Grapefruit League',
       'link': '/api/v1/league/115',
       'abbreviation': 'GL'},
      'allStarStatus': 'N'},
     'i

Appears a little too aggregated. So we should probably just focus on `allPlays`

---

!pip install flatten_json

Let's try to flatten the game by using `flatten_json`

In [31]:
import flatten_json
from flatten_json import flatten

In [104]:
is_it_flat = flatten(test_game)

In [105]:
type(is_it_flat)

dict

In [106]:
is_it_flat

{'copyright': 'Copyright 2019 MLB Advanced Media, L.P.  Use of any content on this page acknowledges agreement to the terms posted here http://gdx.mlb.com/components/copyright.txt',
 'allPlays_0_result_type': 'atBat',
 'allPlays_0_result_event': 'Strikeout',
 'allPlays_0_result_eventType': 'strikeout',
 'allPlays_0_result_description': 'Andrew McCutchen strikes out swinging.',
 'allPlays_0_result_rbi': 0,
 'allPlays_0_result_awayScore': 0,
 'allPlays_0_result_homeScore': 0,
 'allPlays_0_about_atBatIndex': 0,
 'allPlays_0_about_halfInning': 'top',
 'allPlays_0_about_inning': 1,
 'allPlays_0_about_startTime': '2019-04-24T22:58:20.000Z',
 'allPlays_0_about_endTime': '2019-04-24T23:13:04.000Z',
 'allPlays_0_about_isComplete': True,
 'allPlays_0_about_isScoringPlay': False,
 'allPlays_0_about_hasReview': False,
 'allPlays_0_about_hasOut': True,
 'allPlays_0_about_captivatingIndex': 14,
 'allPlays_0_count_balls': 3,
 'allPlays_0_count_strikes': 3,
 'allPlays_0_count_outs': 1,
 'allPlays_0_ma

In [39]:
len(is_it_flat)

24307

So, it is a bit too flat. Especially the decisions it makes to flatten 'across' rather than down. for example, we want rows of pitches, not columns of new pitches.  

How about if we use straight `json`.

Also, found `pd.io.json.json_normalize`

In [7]:
import json
import pandas as pd
from pandas.io.json import json_normalize

Let's go back to `allPlays` and try to flatten it.

In [8]:
all_plays = test_game.get('allPlays')

In [9]:
type(all_plays)

list

In [10]:
test_df = json_normalize(all_plays)

In [11]:
type(test_df)

pandas.core.frame.DataFrame

In [12]:
test_df.columns

Index(['about.atBatIndex', 'about.captivatingIndex', 'about.endTime',
       'about.halfInning', 'about.hasOut', 'about.hasReview', 'about.inning',
       'about.isComplete', 'about.isScoringPlay', 'about.startTime',
       'actionIndex', 'atBatIndex', 'count.balls', 'count.outs',
       'count.strikes', 'matchup.batSide.code', 'matchup.batSide.description',
       'matchup.batter.fullName', 'matchup.batter.id', 'matchup.batter.link',
       'matchup.batterHotColdZoneStats.stats', 'matchup.batterHotColdZones',
       'matchup.pitchHand.code', 'matchup.pitchHand.description',
       'matchup.pitcher.fullName', 'matchup.pitcher.id',
       'matchup.pitcher.link', 'matchup.pitcherHotColdZoneStats.stats',
       'matchup.pitcherHotColdZones', 'matchup.splits.batter',
       'matchup.splits.menOnBase', 'matchup.splits.pitcher', 'pitchIndex',
       'playEndTime', 'playEvents', 'result.awayScore', 'result.description',
       'result.event', 'result.eventType', 'result.homeScore', 'result.rb

In [13]:
# each is list of pitch indices, presumable
test_df['pitchIndex'].tail()

75                   [0]
76          [1, 2, 3, 4]
77    [1, 2, 3, 4, 5, 6]
78             [0, 1, 2]
79    [1, 2, 3, 4, 5, 6]
Name: pitchIndex, dtype: object

In [43]:
test_df.head()

about.atBatIndex  about.captivatingIndex             about.endTime  \
0                 0                      14  2019-04-24T23:13:04.000Z   
1                 1                      34  2019-04-24T23:15:53.000Z   
2                 2                      34  2019-04-24T23:17:43.000Z   
3                 3                       0  2019-04-24T23:21:06.000Z   
4                 4                       0  2019-04-24T23:21:54.000Z   

  about.halfInning  about.hasOut  about.hasReview  about.inning  \
0              top          True            False             1   
1              top         False            False             1   
2              top         False            False             1   
3              top         False            False             1   
4              top          True            False             1   

   about.isComplete  about.isScoringPlay           about.startTime  \
0              True                False  2019-04-24T22:58:20.000Z   
1              True                False  2019-04-24T23:13:05.000Z   
2              True                 True  2019-04-24T23:15:55.000Z   
3              True                False  2019-04-24T23:17:45.000Z   
4              True                False  2019-04-24T23:21:08.000Z   

                         ...                          \
0                        ...                           
1                        ...                           
2                        ...                           
3                        ...                           
4                        ...                           

                                          playEvents  result.awayScore  \
0  [{'details': {'call': {'code': 'B', 'descripti...                 0   
1  [{'details': {'call': {'code': 'S', 'descripti...                 0   
2  [{'details': {'call': {'code': 'B', 'descripti...                 1   
3  [{'details': {'call': {'code': 'B', 'descripti...                 1   
4  [{'details': {'call': {'code': 'X', 'descripti...                 1   

                                  result.description  result.event  \
0             Andrew McCutchen strikes out swinging.     Strikeout   
1  J.T. Realmuto doubles (4) on a sharp line driv...        Double   
2  Bryce Harper doubles (7) on a sharp line drive...        Double   
3                                Rhys Hoskins walks.          Walk   
4  Maikel Franco flies out to right fielder Micha...        Flyout   

   result.eventType result.homeScore result.rbi result.type  runnerIndex  \
0         strikeout                0          0       atBat          [0]   
1            double                0          0       atBat          [0]   
2            double                0          1       atBat    [0, 1, 2]   
3              walk                0          0       atBat          [0]   
4         field_out                0          0       atBat          [0]   

                                             runners  
0  [{'movement': {'start': None, 'end': None, 'ou...  
1  [{'movement': {'start': None, 'end': '2B', 'ou...  
2  [{'movement': {'start': '2B', 'end': '3B', 'ou...  
3  [{'movement': {'start': None, 'end': '1B', 'ou...  
4  [{'movement': {'start': None, 'end': None, 'ou...  

[5 rows x 44 columns]

In [45]:
test_df.iloc[:,:11].head()

about.atBatIndex  about.captivatingIndex             about.endTime  \
0                 0                      14  2019-04-24T23:13:04.000Z   
1                 1                      34  2019-04-24T23:15:53.000Z   
2                 2                      34  2019-04-24T23:17:43.000Z   
3                 3                       0  2019-04-24T23:21:06.000Z   
4                 4                       0  2019-04-24T23:21:54.000Z   

  about.halfInning  about.hasOut  about.hasReview  about.inning  \
0              top          True            False             1   
1              top         False            False             1   
2              top         False            False             1   
3              top         False            False             1   
4              top          True            False             1   

   about.isComplete  about.isScoringPlay           about.startTime actionIndex  
0              True                False  2019-04-24T22:58:20.000Z          []  
1              True                False  2019-04-24T23:13:05.000Z          []  
2              True                 True  2019-04-24T23:15:55.000Z          []  
3              True                False  2019-04-24T23:17:45.000Z          []  
4              True                False  2019-04-24T23:21:08.000Z          []

In [48]:
test_df.iloc[:,11:22].head()

atBatIndex  count.balls  count.outs  count.strikes matchup.batSide.code  \
0           0            3           1              3                    R   
1           1            2           1              2                    R   
2           2            2           1              0                    L   
3           3            4           1              2                    R   
4           4            0           2              0                    R   

  matchup.batSide.description matchup.batter.fullName  matchup.batter.id  \
0                       Right        Andrew McCutchen             457705   
1                       Right           J.T. Realmuto             592663   
2                        Left            Bryce Harper             547180   
3                       Right            Rhys Hoskins             656555   
4                       Right           Maikel Franco             596748   

     matchup.batter.link matchup.batterHotColdZoneStats.stats  \
0  /api/v1/people/457705                                  NaN   
1  /api/v1/people/592663                                  NaN   
2  /api/v1/people/547180                                  NaN   
3  /api/v1/people/656555                                  NaN   
4  /api/v1/people/596748                                  NaN   

  matchup.batterHotColdZones  
0                         []  
1                         []  
2                         []  
3                         []  
4                         []

In [49]:
test_df.iloc[:,22:33].head()

matchup.pitchHand.code matchup.pitchHand.description  \
0                      L                          Left   
1                      L                          Left   
2                      L                          Left   
3                      L                          Left   
4                      L                          Left   

  matchup.pitcher.fullName  matchup.pitcher.id   matchup.pitcher.link  \
0             Jason Vargas              450306  /api/v1/people/450306   
1             Jason Vargas              450306  /api/v1/people/450306   
2             Jason Vargas              450306  /api/v1/people/450306   
3             Jason Vargas              450306  /api/v1/people/450306   
4             Jason Vargas              450306  /api/v1/people/450306   

  matchup.pitcherHotColdZoneStats.stats matchup.pitcherHotColdZones  \
0                                   NaN                          []   
1                                   NaN                          []   
2                                   NaN                          []   
3                                   NaN                          []   
4                                   NaN                          []   

  matchup.splits.batter matchup.splits.menOnBase matchup.splits.pitcher  \
0                vs_LHP                    Empty                 vs_RHB   
1                vs_LHP                     RISP                 vs_RHB   
2                vs_LHP                     RISP                 vs_LHB   
3                vs_LHP                     RISP                 vs_RHB   
4                vs_LHP                     RISP                 vs_RHB   

                 pitchIndex  
0        [0, 1, 2, 3, 4, 5]  
1  [0, 1, 2, 3, 4, 5, 6, 7]  
2                 [0, 1, 2]  
3     [0, 1, 2, 3, 4, 5, 6]  
4                       [0]

In [62]:
test_df.iloc[:,33:44].head()

playEndTime  \
0  2019-04-24T23:13:04.000Z   
1  2019-04-24T23:15:53.000Z   
2  2019-04-24T23:17:43.000Z   
3  2019-04-24T23:21:06.000Z   
4  2019-04-24T23:21:54.000Z   

                                          playEvents  result.awayScore  \
0  [{'details': {'call': {'code': 'B', 'descripti...                 0   
1  [{'details': {'call': {'code': 'S', 'descripti...                 0   
2  [{'details': {'call': {'code': 'B', 'descripti...                 1   
3  [{'details': {'call': {'code': 'B', 'descripti...                 1   
4  [{'details': {'call': {'code': 'X', 'descripti...                 1   

                                  result.description result.event  \
0             Andrew McCutchen strikes out swinging.    Strikeout   
1  J.T. Realmuto doubles (4) on a sharp line driv...       Double   
2  Bryce Harper doubles (7) on a sharp line drive...       Double   
3                                Rhys Hoskins walks.         Walk   
4  Maikel Franco flies out to right fielder Micha...       Flyout   

  result.eventType  result.homeScore  result.rbi result.type runnerIndex  \
0        strikeout                 0           0       atBat         [0]   
1           double                 0           0       atBat         [0]   
2           double                 0           1       atBat   [0, 1, 2]   
3             walk                 0           0       atBat         [0]   
4        field_out                 0           0       atBat         [0]   

                                             runners  
0  [{'movement': {'start': None, 'end': None, 'ou...  
1  [{'movement': {'start': None, 'end': '2B', 'ou...  
2  [{'movement': {'start': '2B', 'end': '3B', 'ou...  
3  [{'movement': {'start': None, 'end': '1B', 'ou...  
4  [{'movement': {'start': None, 'end': None, 'ou...

Having inspected these columns, and nested dicts, think have captured everything obviously useful for our business question(s).

In [67]:
test_df.shape

(80, 44)

## Prototype for data pull

In [68]:
list_for_new_df = []

# Data from allPlays
ap_sel_cols = ['about.atBatIndex', 'matchup.batSide.code', 'matchup.pitchHand.code', 'count.balls'
              ,'count.strikes', 'count.outs']

# Data from playEvents
plev_sel_cols = ['details.type.code', 'details.type.description', 
            'details.call.code', 'details.call.description', 
            'details.isBall', 'isPitch', 'details.isStrike'
            ,'pitchData.breaks.breakAngle'
            ,'pitchData.breaks.breakLength', 'pitchData.breaks.breakY'
            ,'pitchData.breaks.spinDirection', 'pitchData.breaks.spinRate'
            ,'pitchData.coordinates.aX'
            , 'pitchData.coordinates.aY','pitchData.coordinates.aZ', 'pitchData.coordinates.pX'
            , 'pitchData.coordinates.pZ', 'pitchData.coordinates.pfxX', 'pitchData.coordinates.pfxZ'
            , 'pitchData.coordinates.vX0', 'pitchData.coordinates.vY0', 'pitchData.coordinates.vZ0'
            , 'pitchData.coordinates.x', 'pitchData.coordinates.x0', 'pitchData.coordinates.y'
            , 'pitchData.coordinates.y0','pitchData.coordinates.z0', 'pitchData.endSpeed'
            , 'pitchData.startSpeed', 'pitchNumber'
           ]

# Now go through each row. If there is nested list, json_normalize it
#for index, row in test_df.head(2).iterrows(): #Just using first 2 rows for testing
for index, row in test_df.iterrows(): #Just using first 2 rows for testing
    
    # saw playEvents is a nested list, so json_normalize it
    play_events_df = json_normalize(row['playEvents'])
    
    # look at runners
    runners_df = json_normalize(row['runners'])
        
    # Loop through THIS NESTED dataframe and NOW build the row for the new df    
    for plev_ind, plev_row in play_events_df.iterrows():
  
        # Instantiate new dict, which will be a single row in target df
        curr_dict = {}
    
#         # atBatIndex
#         curr_dict['at_bat_index'] = row['about.atBatIndex']

#         # batside
#         curr_dict['bat_side'] = row['matchup.batSide.code']

#         # pitchHand
#         curr_dict['pitch_hand'] = row['matchup.pitchHand.code']
        for col_ap in ap_sel_cols:
            curr_dict[col_ap] = row[col_ap]
    
        for col_plev in plev_sel_cols:
            curr_dict[col_plev] = plev_row[col_plev]
        
#         curr_dict['pitch_label_cd'] = pev_row['details.type.code']
#         curr_dict['pitch_label_desc'] = pev_row['details.type.description']
        
        # collect row dictionary into list
        list_for_new_df.append(curr_dict)



Let's look at the last iteration of `play_events_df`

In [69]:
play_events_df.columns

Index(['battingOrder', 'count.balls', 'count.outs', 'count.strikes',
       'details.awayScore', 'details.ballColor', 'details.call.code',
       'details.call.description', 'details.code', 'details.description',
       'details.event', 'details.eventType', 'details.hasReview',
       'details.homeScore', 'details.isBall', 'details.isInPlay',
       'details.isScoringPlay', 'details.isStrike', 'details.trailColor',
       'details.type.code', 'details.type.description', 'endTime', 'index',
       'isPitch', 'pfxId', 'pitchData.breaks.breakAngle',
       'pitchData.breaks.breakLength', 'pitchData.breaks.breakY',
       'pitchData.breaks.spinDirection', 'pitchData.breaks.spinRate',
       'pitchData.coordinates.aX', 'pitchData.coordinates.aY',
       'pitchData.coordinates.aZ', 'pitchData.coordinates.pX',
       'pitchData.coordinates.pZ', 'pitchData.coordinates.pfxX',
       'pitchData.coordinates.pfxZ', 'pitchData.coordinates.vX0',
       'pitchData.coordinates.vY0', 'pitchData.coordin

Let's look at the last iteration of `runners_df`

In [71]:
runners_df.head().T

0
credits                     [{'player': {'id': 592663, 'link': '/api/v1/pe...
details.earned                                                          False
details.event                                                       Strikeout
details.eventType                                                   strikeout
details.isScoringEvent                                                  False
details.movementReason                                                   None
details.playIndex                                                           6
details.rbi                                                             False
details.responsiblePitcher                                               None
details.runner.fullName                                            J.D. Davis
details.runner.id                                                      605204
details.runner.link                                     /api/v1/people/605204
details.teamUnearned                                                    False
movement.end                                                             None
movement.isOut                                                           True
movement.outBase                                                           1B
movement.outNumber                                                          3
movement.start                                                           None

Nothing here looks immediately useful.

In [72]:
len(list_for_new_df)

365

Check all the selected fields make sense.

In [73]:
list_for_new_df[:1]

[{'about.atBatIndex': 0,
  'matchup.batSide.code': 'R',
  'matchup.pitchHand.code': 'L',
  'count.balls': 3,
  'count.strikes': 3,
  'count.outs': 1,
  'details.type.code': 'FF',
  'details.type.description': 'Four-Seam Fastball',
  'details.call.code': 'B',
  'details.call.description': 'Ball - Called',
  'details.isBall': True,
  'isPitch': True,
  'details.isStrike': False,
  'pitchData.breaks.breakAngle': 36.0,
  'pitchData.breaks.breakLength': 4.8,
  'pitchData.breaks.breakY': 24.0,
  'pitchData.breaks.spinDirection': 146,
  'pitchData.breaks.spinRate': 2303,
  'pitchData.coordinates.aX': 12.55,
  'pitchData.coordinates.aY': 26.39,
  'pitchData.coordinates.aZ': -13.99,
  'pitchData.coordinates.pX': 1.48,
  'pitchData.coordinates.pZ': 1.58,
  'pitchData.coordinates.pfxX': 8.38,
  'pitchData.coordinates.pfxZ': 12.13,
  'pitchData.coordinates.vX0': -5.79,
  'pitchData.coordinates.vY0': -122.28,
  'pitchData.coordinates.vZ0': -6.67,
  'pitchData.coordinates.x': 60.67,
  'pitchData.coo

Seems legit.

In [75]:
# Proof of concept on target dataframe
new_df = pd.DataFrame(list_for_new_df)

new_df.head(10)

about.atBatIndex  count.balls  count.outs  count.strikes details.call.code  \
0                 0            3           1              3                 B   
1                 0            3           1              3                 B   
2                 0            3           1              3                 S   
3                 0            3           1              3                 B   
4                 0            3           1              3                 S   
5                 0            3           1              3                 S   
6                 1            2           1              2                 S   
7                 1            2           1              2                 B   
8                 1            2           1              2                 S   
9                 1            2           1              2                 S   

  details.call.description details.isBall details.isStrike details.type.code  \
0            Ball - Called           True            False                FF   
1            Ball - Called           True            False                FT   
2        Strike - Swinging          False             True                FT   
3            Ball - Called           True            False                CH   
4        Strike - Swinging          False             True                FT   
5        Strike - Swinging          False             True                FF   
6        Strike - Swinging          False             True                FF   
7            Ball - Called           True            False                FF   
8        Strike - Swinging          False             True                FT   
9        Strike - Swinging          False             True                FF   

  details.type.description     ...       pitchData.coordinates.vY0  \
0       Four-Seam Fastball     ...                         -122.28   
1        Two-Seam Fastball     ...                         -122.19   
2        Two-Seam Fastball     ...                         -123.08   
3                 Changeup     ...                         -116.93   
4        Two-Seam Fastball     ...                         -123.69   
5       Four-Seam Fastball     ...                         -126.23   
6       Four-Seam Fastball     ...                         -123.85   
7       Four-Seam Fastball     ...                         -124.61   
8        Two-Seam Fastball     ...                         -124.14   
9       Four-Seam Fastball     ...                         -126.17   

  pitchData.coordinates.vZ0 pitchData.coordinates.x  pitchData.coordinates.x0  \
0                     -6.67                   60.67                      2.80   
1                     -3.37                   55.45                      2.74   
2                     -5.83                  125.65                      2.60   
3                     -5.93                   96.61                      2.48   
4                     -3.51                  122.61                      2.55   
5                     -4.25                   62.49                      2.80   
6                     -6.53                  106.50                      2.58   
7                     -2.50                  147.61                      2.32   
8                     -2.75                   68.10                      2.62   
9                     -1.25                  140.82                      2.52   

   pitchData.coordinates.y  pitchData.coordinates.y0  \
0                   196.16                      50.0   
1                   170.30                      50.0   
2                   193.72                      50.0   
3                   206.66                      50.0   
4                   167.13                      50.0   
5                   173.25                      50.0   
6                   195.08                      50.0   
7                   147.00                      50.0   
8                   159.87                      50.0   
9      

Looks good!

In [77]:
new_df.shape

(365, 36)

In [78]:
new_df.tail()

about.atBatIndex  count.balls  count.outs  count.strikes  \
360                79            3           3              3   
361                79            3           3              3   
362                79            3           3              3   
363                79            3           3              3   
364                79            3           3              3   

    details.call.code details.call.description details.isBall  \
360                 B            Ball - Called           True   
361                 B            Ball - Called           True   
362                 S        Strike - Swinging          False   
363                 B            Ball - Called           True   
364                 S        Strike - Swinging          False   

    details.isStrike details.type.code details.type.description     ...       \
360            False                SL                   Slider     ...        
361            False                SL                   Slider     ...        
362             True                FF       Four-Seam Fastball     ...        
363            False                SL                   Slider     ...        
364             True                FF       Four-Seam Fastball     ...        

     pitchData.coordinates.vY0 pitchData.coordinates.vZ0  \
360                    -125.24                     -6.89   
361                    -126.37                     -8.23   
362                    -133.58                     -6.46   
363                    -125.90                     -7.55   
364                    -134.17                     -4.97   

    pitchData.coordinates.x  pitchData.coordinates.x0  \
360                   44.66                     -1.80   
361                  125.22                     -2.16   
362                   91.21                     -1.75   
363                   91.17                     -1.81   
364                  119.54                     -1.91   

     pitchData.coordinates.y  pitchData.coordinates.y0  \
360                   209.73                      50.0   
361                   219.16                      50.0   
362                   174.95                      50.0   
363                   218.02                      50.0   
364                   155.35                      50.0   

     pitchData.coordinates.z0  pitchData.endSpeed  pitchData.startSpeed  \
360                      6.01                79.7                  86.3   
361                      5.96                80.2                  87.0   
362                      6.00                84.1                  92.0   
363                      5.89                80.3                  86.7   
364                      5.88                84.1                  92.3   

     pitchNumber  
360          2.0  
361          3.0  
362          4.0  
363          5.0  
364          6.0  

[5 rows x 36 columns]

In [79]:
new_df.columns

Index(['about.atBatIndex', 'count.balls', 'count.outs', 'count.strikes',
       'details.call.code', 'details.call.description', 'details.isBall',
       'details.isStrike', 'details.type.code', 'details.type.description',
       'isPitch', 'matchup.batSide.code', 'matchup.pitchHand.code',
       'pitchData.breaks.breakAngle', 'pitchData.breaks.breakLength',
       'pitchData.breaks.breakY', 'pitchData.breaks.spinDirection',
       'pitchData.breaks.spinRate', 'pitchData.coordinates.aX',
       'pitchData.coordinates.aY', 'pitchData.coordinates.aZ',
       'pitchData.coordinates.pX', 'pitchData.coordinates.pZ',
       'pitchData.coordinates.pfxX', 'pitchData.coordinates.pfxZ',
       'pitchData.coordinates.vX0', 'pitchData.coordinates.vY0',
       'pitchData.coordinates.vZ0', 'pitchData.coordinates.x',
       'pitchData.coordinates.x0', 'pitchData.coordinates.y',
       'pitchData.coordinates.y0', 'pitchData.coordinates.z0',
       'pitchData.endSpeed', 'pitchData.startSpeed', 'pitchNu

### ** OK, think we have enough here. Going to pull cleaned up version into new workbook to act as a pseudo-template for work moving forward.**

-----

-----

# Old or Abandoned Ideas

In [49]:
yyy.head()

battingOrder                       count  \
0          NaN  {'balls': 1, 'strikes': 0}   
1          NaN  {'balls': 2, 'strikes': 0}   
2          NaN  {'balls': 2, 'strikes': 1}   
3          NaN  {'balls': 3, 'strikes': 1}   
4          NaN  {'balls': 3, 'strikes': 2}   

                                             details  \
0  {'call': {'code': 'B', 'description': 'Ball - ...   
1  {'call': {'code': 'B', 'description': 'Ball - ...   
2  {'call': {'code': 'S', 'description': 'Strike ...   
3  {'call': {'code': 'B', 'description': 'Ball - ...   
4  {'call': {'code': 'S', 'description': 'Strike ...   

                    endTime hitData  index  isPitch          pfxId  \
0  2019-04-24T23:12:04.000Z     NaN      0     True  190424_231125   
1  2019-04-24T23:12:17.000Z     NaN      1     True  190424_231209   
2  2019-04-24T23:12:30.000Z     NaN      2     True  190424_231222   
3  2019-04-24T23:12:43.000Z     NaN      3     True  190424_231236   
4  2019-04-24T23:12:59.000Z     NaN      4     True  190424_231248   

                                           pitchData  pitchNumber  \
0  {'startSpeed': 84.3, 'endSpeed': 76.4, 'strike...          1.0   
1  {'startSpeed': 84.2, 'endSpeed': 76.3, 'strike...          2.0   
2  {'startSpeed': 84.9, 'endSpeed': 77.0, 'strike...          3.0   
3  {'startSpeed': 80.8, 'endSpeed': 73.4, 'strike...          4.0   
4  {'startSpeed': 85.2, 'endSpeed': 78.0, 'strike...          5.0   

                                 playId player position  \
0  e7c6b8db-d32a-4592-a7ba-7747fc091a6a    NaN      NaN   
1  4903682e-ec48-4360-9c75-3ec89bf7f15d    NaN      NaN   
2  8c8d56a6-0bc7-4ee1-b3a1-bcea95d4ce64    NaN      NaN   
3  37f3f56e-f1d4-4131-b82c-6c88c4d009dc    NaN      NaN   
4  3dc32a07-e2bf-4b6f-8712-5b16ee33b1d5    NaN      NaN   

                  startTime   type  
0  2019-04-24T23:11:20.000Z  pitch  
1  2019-04-24T23:12:04.000Z  pitch  
2  2019-04-24T23:12:17.000Z  pitch  
3  2019-04-24T23:12:30.000Z  pitch  
4  2019-04-24T23:12:43.000Z  pitch

In [50]:
yyy.columns

Index(['battingOrder', 'count', 'details', 'endTime', 'hitData', 'index',
       'isPitch', 'pfxId', 'pitchData', 'pitchNumber', 'playId', 'player',
       'position', 'startTime', 'type'],
      dtype='object')

In [58]:
def only_dict(d):
    '''
    Convert json string representation of dictionary to a python dict
    '''
    return json.loads(d)

In [59]:
def list_of_dicts(ld):
    '''
    Create a mapping of the tuples formed after 
    converting json strings of list to a python list   
    '''
    return dict([(list(d.values())[1], list(d.values())[0]) for d in json.loads(ld)])


In [ ]:
A = json_normalize(df['columnA'].apply(only_dict).tolist()).add_prefix('columnA.')
B = json_normalize(df['columnB'].apply(list_of_dicts).tolist()).add_prefix('columnB.pos.') 

In [ ]:
A = json_normalize(df['columnA'].apply(only_dict).tolist()).add_prefix('columnA.')
B = json_normalize(df['columnB'].apply(list_of_dicts).tolist()).add_prefix('columnB.pos.') 

In [62]:
aaa = json_normalize(yyy['pitchData'].tolist()).add_prefix('col_pitch_data.')


AttributeError: 'float' object has no attribute 'values'

In [60]:
yyy.head(2)

battingOrder                       count  \
0          NaN  {'balls': 1, 'strikes': 0}   
1          NaN  {'balls': 2, 'strikes': 0}   

                                             details  \
0  {'call': {'code': 'B', 'description': 'Ball - ...   
1  {'call': {'code': 'B', 'description': 'Ball - ...   

                    endTime hitData  index  isPitch          pfxId  \
0  2019-04-24T23:12:04.000Z     NaN      0     True  190424_231125   
1  2019-04-24T23:12:17.000Z     NaN      1     True  190424_231209   

                                           pitchData  pitchNumber  \
0  {'startSpeed': 84.3, 'endSpeed': 76.4, 'strike...          1.0   
1  {'startSpeed': 84.2, 'endSpeed': 76.3, 'strike...          2.0   

                                 playId player position  \
0  e7c6b8db-d32a-4592-a7ba-7747fc091a6a    NaN      NaN   
1  4903682e-ec48-4360-9c75-3ec89bf7f15d    NaN      NaN   

                  startTime   type  
0  2019-04-24T23:11:20.000Z  pitch  
1  2019-04-24T23:12:04.000Z  pitch

In [75]:
pitch_data_example = yyy.loc[yyy.index[0],'pitchData']

In [76]:
type(pitch_data_example)

dict

In [79]:
json_normalize(pitch_data_example)

breaks.breakAngle  breaks.breakLength  breaks.breakY  breaks.spinDirection  \
0               36.0                 4.8           24.0                   146   

   breaks.spinRate  coordinates.aX  coordinates.aY  coordinates.aZ  \
0             2303           12.55           26.39          -13.99   

   coordinates.pX  coordinates.pZ  ...   coordinates.x  coordinates.x0  \
0            1.48            1.58  ...           60.67             2.8   

   coordinates.y  coordinates.y0  coordinates.z0  endSpeed  startSpeed  \
0         196.16            50.0            5.57      76.4        84.3   

   strikeZoneBottom  strikeZoneTop  zone  
0              1.66           3.42    14  

[1 rows x 25 columns]

---

---

---

## Look at `mlbgame` module

In [7]:
import mlbgame

In [15]:
from __future__ import print_function
import mlbgame

#month = mlbgame.games(2015, 6, home='Mets')
month = mlbgame.games(2015, 6, 1)

In [16]:
month

[[<mlbgame.game.GameScoreboard at 0x11be37748>,

In [21]:
type(month)

list

In [8]:
games = mlbgame.combine_games(month)
for game in games:
    print(game)

KeyboardInterrupt: 

In [14]:
from __future__ import print_function
import mlbgame

day = mlbgame.day(2015, 4, 12, home='Royals', away='Royals')
game = day[0]
output = 'Winning pitcher: %s (%s) - Losing Pitcher: %s (%s)'
print(output % (game.w_pitcher, game.w_team, game.l_pitcher, game.l_team))

Winning pitcher: Yordano Ventura (Royals) - Losing Pitcher: C.J. Wilson (Angels)


In [9]:
import pandas as pd

This ended up not going much abandoned. Using `statsapi` still for now.  
(wm 2019.05.26)